In [0]:
import pyspark.sql.functions as F

malaysia_df = spark.table('hive_metastore.google_drive._2021_violence_against_women_cases_malasia')
malaysia_df.limit(20).display()

_line,_fivetran_synced,year,type_of_case,sex,number_of_violence
22,2025-01-25T06:25:31.336Z,2021,Rape,Women,1360
59,2025-01-25T06:25:31.336Z,2021,Unnatural,Men,73
27,2025-01-25T06:25:31.336Z,2017,Incest,Men,0
64,2025-01-25T06:25:31.337Z,2018,Sexual disoder,Women,253
32,2025-01-25T06:25:31.336Z,2020,Incest,Women,259
69,2025-01-25T06:25:31.337Z,2020,Sexual disoder,Men,24
0,2025-01-25T06:25:31.335Z,2016,Domestic violence,Women,4285
37,2025-01-25T06:25:31.336Z,2016,Molest,Men,38
74,2025-01-25T06:25:31.337Z,2017,Distribute/ display pornographic material,Women,35
5,2025-01-25T06:25:31.335Z,2018,Domestic violence,Men,1474


filter female victims

In [0]:
malaysia_df_filtered = malaysia_df.where(
    F.col('sex') == 'Women',
).withColumn(
    'country', F.lit('malaysia'),
).select(
    F.col('country'),
    F.col('year'),
    F.col('type_of_case').alias('type_of_violence'),
    F.col('number_of_violence').alias('total_cases'),
)
malaysia_df_filtered.limit(20).display()

country,year,type_of_violence,total_cases
malaysia,2021,Rape,1360
malaysia,2018,Sexual disoder,253
malaysia,2020,Incest,259
malaysia,2016,Domestic violence,4285
malaysia,2017,Distribute/ display pornographic material,35
malaysia,2019,Molest,1064
malaysia,2021,Domestic violence,5131
malaysia,2018,Unnatural,159
malaysia,2020,Rape,1378
malaysia,2017,Sexual disoder,226


sum all types

In [0]:
malaysa_df_any = malaysia_df_filtered.groupBy('year', 'country').agg(
    F.sum('total_cases').alias('total_cases')
).withColumn(
    'type_of_violence', F.lit('any')
).select(
    F.col('country'),
    F.col('year'),
    F.col('type_of_violence'),
    F.col('total_cases')
)
malaysia_df_any_violence = malaysia_df_filtered.union(malaysa_df_any)
malaysia_df_any_violence.limit(20).display()

country,year,type_of_violence,total_cases
malaysia,2021,Rape,1360
malaysia,2018,Sexual disoder,253
malaysia,2020,Incest,259
malaysia,2016,Domestic violence,4285
malaysia,2017,Distribute/ display pornographic material,35
malaysia,2019,Molest,1064
malaysia,2021,Domestic violence,5131
malaysia,2018,Unnatural,159
malaysia,2020,Rape,1378
malaysia,2017,Sexual disoder,226


now we need to delete the '_' character form the year